In [1]:
! pip install datasets transformers seqeval

     |████████████████████████████████| 325 kB 30.2 MB/s 
     |████████████████████████████████| 4.0 MB 67.8 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |████████████████████████████████| 77 kB 1.8 MB/s 
     |████████████████████████████████| 136 kB 10.4 MB/s 
     |████████████████████████████████| 1.1 MB 62.4 MB/s 
     |████████████████████████████████| 212 kB 72.2 MB/s 
     |████████████████████████████████| 127 kB 75.4 MB/s 
     |████████████████████████████████| 6.6 MB 55.3 MB/s 
     |████████████████████████████████| 895 kB 53.2 MB/s 
     |████████████████████████████████| 596 kB 59.2 MB/s 
     |████████████████████████████████| 144 kB 72.0 MB/s 
     |████████████████████████████████| 271 kB 76.5 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=1e3c939f7dac0c491571f8be1bdb1f542518a77502c15747566138ee4d25bcd9
  Stored in directory: /root/.cache/pip/wh

In [2]:
from pathlib import Path
from argparse import Namespace
from typing import Union, List
from fastprogress import progress_bar
from typing_extensions import TypedDict
import torch
from datasets import Dataset, DatasetDict, load_metric
import numpy as np
from transformers import DataCollatorForTokenClassification, AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer


In [3]:
data_dir = Path("/content/drive/Shareddrives/derzart@gmail.com/studies/nlp-final-project/data")

args = Namespace(
    batch_size=64,
    num_workers=4
)

train_data_path = data_dir / '%-training'
valid_data_path = data_dir / '%-dev'
test_data_path = data_dir / '%-test'

In [4]:
SYMBOL_DICT = {
    "COMMA": ",",
}

LABEL_LIST = ["NONE", "PRED", "ARG1", "SUPPORT"]
POS_LIST = ["CC", "CD", "DT", "FW", "IN", "JJ", "JJR", "JJS", 
            "LS", "MD", "NN", "NNS", "NNP", "NNPS", "PDT", 
            "POS", "PRP", "PRP$", "RB", "RBR", "RBS", "SYM", 
            "TO", "UH", "VB", "VBD", "VBG", "VBN", "VBP",
            "VBZ", "WDT", "WP", "WP$", "WRB", "PU", "EX", 
            "RP"]
BIO_TAG_CONVERSION_DICT = {
    ".": "PU",
    ",": "PU",
    "COMMA": "PU",
    "$": "PU",
    ":": "PU",
    "(": "PU",
    ")": "PU",
    "``": "PU",
    "''": "PU",
    "#": "PU"
}
BIO_TAG_LIST = ["O", "B-NP", "I-NP", "B-VP", "I-VP", "B-PP",
                "I-PP", "B-ADJP", "I-ADJP", "B-ADVP", "I-ADVP",
                "B-SBAR", "I-SBAR", "B-PRT", "I-PRT", "B-CONJP",
                "I-CONJP", "B-UCP", "I-UCP"]

In [5]:
class Word(TypedDict):
  word: str
  pos: str
  biotag: str
  label: Union[str, None]

def parse_input(input_file: Union[str, Path], drop_label = False) -> List[Union[List[Word], None]]:
    """
    Parses the input file and returns a list of lists of words.
    """
    with open(input_file, "r") as f:
        lines = f.readlines()
    sentences: List[Union[List[Word], None]] = []
    last_sentence: List[Word] = []
    print("Parsing input file lines...")
    line_no = 0
    for line in progress_bar(lines):
        line_no += 1
        line = line.strip()
        word_info = line.split("\t")
        if len(word_info) >= 5:
            word_str = word_info[0].strip()
            if word_str in SYMBOL_DICT:
              word_str = SYMBOL_DICT[word_str]
            pos = word_info[1].strip()
            if pos in BIO_TAG_CONVERSION_DICT:
              pos = BIO_TAG_CONVERSION_DICT[pos]
            if pos not in POS_LIST:
              print(f"Warning: invalid POS on line {line_no} \"{pos}\", treated as PU.")
              pos = "PU"
            biotag = word_info[2].strip()
            if biotag not in BIO_TAG_LIST:
              print(f"Warning: invalid bio tag on line {line_no} \"{biotag}\", treated as O.")
              biotag = "O"
            if len(word_info) >= 6:
                label = word_info[5].strip()
            else:
                label = "NONE"
            if label not in LABEL_LIST:
              print(f"Warning: invalid label on line {line_no} \"{label}\", treated as NONE.")
              label = "NONE"
            if drop_label:
              label = None
            word = Word(word=word_str, pos=pos, biotag=biotag, label=label)
            last_sentence.append(word)
        else:
            if len(last_sentence) > 0:
                sentences.append(last_sentence)
            last_sentence = []
    if len(last_sentence) > 0:
        sentences.append(last_sentence)
    return sentences

In [6]:
train_sentences = parse_input(train_data_path)
valid_sentences = parse_input(valid_data_path)
test_sentences = parse_input(test_data_path, drop_label=True)

Parsing input file lines...


Parsing input file lines...


Parsing input file lines...


In [7]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=len(LABEL_LIST))
# tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
# model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english", num_labels=len(LABEL_LIST))

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

In [8]:
def build_dataset_from_sentences(sentences, drop_label = False):
  dataset_tokens = []
  dataset_partitive_roles = []
  dataset_pos_tags = []
  dataset_bio_tags = []
  for sentence in sentences:
    tokens = [word['word'] for word in sentence]
    if not drop_label:
      partitive_roles = [LABEL_LIST.index(word['label']) for word in sentence]
    else:
      partitive_roles = None
    pos_tags = [POS_LIST.index(word['pos']) for word in sentence]
    bio_tags = [BIO_TAG_LIST.index(word['biotag']) for word in sentence]
    dataset_tokens.append(tokens)
    if not drop_label:
      dataset_partitive_roles.append(partitive_roles)
    dataset_pos_tags.append(pos_tags)
    dataset_bio_tags.append(bio_tags)
  if not drop_label:
    dataset_dict = {
        "tokens": dataset_tokens,
        "partitive_roles": dataset_partitive_roles,
        "pos_tags": dataset_pos_tags,
        "bio_tags": dataset_bio_tags
    }
  else:
    dataset_dict = {
        "tokens": dataset_tokens,
        "pos_tags": dataset_pos_tags,
        "bio_tags": dataset_bio_tags
    }
  return Dataset.from_dict(dataset_dict)

In [9]:
train_raw_dataset = build_dataset_from_sentences(train_sentences)
valid_raw_dataset = build_dataset_from_sentences(valid_sentences)
test_raw_dataset = build_dataset_from_sentences(test_sentences, drop_label=True)

raw_datasets = DatasetDict(train=train_raw_dataset, valid=valid_raw_dataset, test=test_raw_dataset)

In [10]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], is_split_into_words=True)

    if "partitive_roles" in examples:
      labels = []
      for i, label in enumerate(examples["partitive_roles"]):
          word_ids = tokenized_inputs.word_ids(batch_index=i)
          previous_word_idx = None
          label_ids = []
          for word_idx in word_ids:
              # Special tokens have a word id that is None. We set the label to -100 so they are automatically
              # ignored in the loss function.
              if word_idx is None:
                  label_ids.append(-100)
              # We set the label for the first token of each word.
              elif word_idx != previous_word_idx:
                  label_ids.append(label[word_idx])
              # For the other tokens in a word, we set the label to either the current label or -100, depending on
              # the label_all_tokens flag.
              else:
                  label_ids.append(label[word_idx] if label_all_tokens else -100)
              previous_word_idx = word_idx

          labels.append(label_ids)

      tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [11]:
tokenized_datasets = raw_datasets.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [LABEL_LIST[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [LABEL_LIST[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [13]:
batch_size=64

train_args = TrainingArguments(
    "bert_partitive_roles",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01
)

data_collator = DataCollatorForTokenClassification(tokenizer)

metric = load_metric("seqeval")

In [14]:
trainer = Trainer(
    model,
    train_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, bio_tags, pos_tags, partitive_roles. If tokens, bio_tags, pos_tags, partitive_roles are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2367
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 370


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.188428,0.482428,0.304435,0.373300,0.930215
2,No log,0.092251,0.735849,0.786290,0.760234,0.965491
3,No log,0.069205,0.818367,0.808468,0.813387,0.972393
4,No log,0.065502,0.787763,0.830645,0.808636,0.971626
5,No log,0.060821,0.792308,0.830645,0.811024,0.972393
6,No log,0.060109,0.777577,0.866935,0.819828,0.972393
7,No log,0.060240,0.779245,0.832661,0.805068,0.971626
8,No log,0.059150,0.777154,0.836694,0.805825,0.972009
9,No log,0.059127,0.788571,0.834677,0.810970,0.972393
10,No log,0.059291,0.783019,0.836694,0.808967,0.972009


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, bio_tags, pos_tags, partitive_roles. If tokens, bio_tags, pos_tags, partitive_roles are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 83
  Batch size = 64
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NONE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PRED seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: SUPPORT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packag

TrainOutput(global_step=370, training_loss=0.09560032406368771, metrics={'train_runtime': 137.6297, 'train_samples_per_second': 171.983, 'train_steps_per_second': 2.688, 'total_flos': 420538030861776.0, 'train_loss': 0.09560032406368771, 'epoch': 10.0})

In [15]:
test_results = trainer.predict(tokenized_datasets['test'])

The following columns in the test set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, bio_tags, pos_tags. If tokens, bio_tags, pos_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 150
  Batch size = 64


In [23]:
for prediction in test_results.predictions[1]:
  index = np.argmax(prediction)
  if index == 2:
    print(prediction)

[-0.5091784 -3.1036859  4.4681897 -1.5725286]
[ 1.1653163 -2.2387307  1.8922228 -1.7388004]


In [ ]:
np.argmax([1, 5, 3, 5])

1

In [49]:
out = []

for i in range(len(tokenized_datasets['test'])):
  sentence = tokenized_datasets['test'][i]
  tokenized_input = tokenizer(sentence["tokens"], truncation=True, is_split_into_words=True)
  predictions = test_results.predictions[i]
  label_ids = []
  max_arg1_index = -1
  max_arg1_value = -100
  for j, prediction in enumerate(predictions):
    arg1_value = prediction[2]
    if arg1_value > max_arg1_value:
      max_arg1_value = arg1_value
      max_arg1_index = j
  for j, prediction in enumerate(predictions):
    label_ids.append(2 if j == max_arg1_index else 0)
  word_id_to_label_idx = {}
  for j, word_id in enumerate(tokenized_input.word_ids()):
    if word_id in word_id_to_label_idx or word_id is None:
      continue
    word_id_to_label_idx[word_id] = j
  labelings = []
  has_arg1 = False

  for j, token in enumerate(sentence["tokens"]):
    label_idx = word_id_to_label_idx[j]
    next_label_idx = word_id_to_label_idx[j+1] if j+1 < len(word_id_to_label_idx) else len(label_ids)

    is_arg1 = False

    for k in range(label_idx, next_label_idx):
      label_id = label_ids[k]
      if label_id == 2:
        is_arg1 = True

    label = 'ARG1' if is_arg1 else None
    if label == 'ARG1':
      has_arg1 = True
    if token == ',':
      token = 'COMMA'
    labelings.append((token, label))
  out.append(labelings)

  if not has_arg1:
    print(f"line {i} has no arg1")


In [58]:
arg1_count = 0
for labelings in out:
  has_arg1 = 0
  for token, label in labelings:
    if label == 'ARG1':
      has_arg1 += 1
  if has_arg1 == 1:
    arg1_count += 1
arg1_count

150

In [59]:
out_path = data_dir / 'test-out-distilbert-one-arg-test'

In [60]:
arg1_count = 0

with open(out_path, 'w') as f:
  for line in out:
    for labling in line:
      if labling[1] and labling[1] == 'ARG1':
        f.write(f"{labling[0]}\t{labling[1]}\n")
        arg1_count += 1
      else:
        f.write(f"{labling[0]}\n")
    f.write("\n")

arg1_count

150